In [12]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)



Mounted at /content/gdrive


In [ ]:
!nvidia-smi
!rm -rf images
!rm -rf cropped_images/
!unzip images.zip


Streaming output truncated to the last 5000 lines.
  inflating: images/batch_3/6996/6996_287.jpeg  
  inflating: images/batch_3/6996/6996_325.jpeg  
  inflating: images/batch_3/6996/6996_329.jpeg  
  inflating: images/batch_3/6996/6996_298.jpeg  
  inflating: images/batch_3/6996/6996_267.jpeg  
  inflating: images/batch_3/6996/6996_206.jpg  
  inflating: images/batch_3/6996/6996_324.jpeg  
  inflating: images/batch_3/6996/6996_262.jpg  
  inflating: images/batch_3/6996/6996_209.jpeg  
  inflating: images/batch_3/6996/6996_272.jpg  
  inflating: images/batch_3/6996/6996_309.jpeg  
  inflating: images/batch_3/6996/6996_226.jpeg  
  inflating: images/batch_3/6996/6996_218.jpeg  
  inflating: images/batch_3/6996/6996_236.jpeg  
  inflating: images/batch_3/6996/6996_246.jpeg  
  inflating: images/batch_3/6996/6996_199.jpeg  
  inflating: images/batch_3/6996/6996_249.jpg  
  inflating: images/batch_3/6996/6996_303.jpg  
  inflating: images/batch_3/6996/6996_274.jpeg  
  inflating: images/bat

In [13]:

!cp /content/gdrive/MyDrive/images_mushrooms.zip /content/


In [ ]:
!unzip images

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 60.4/235.7 GB disk)


In [ ]:
!yolo task=detect mode=predict model=yolo11n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

100% 5.35M/5.35M [00:00<00:00, 82.5MB/s]
Ultralytics 8.3.40 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

100% 104k/104k [00:00<00:00, 84.6MB/s]
image 1/1 /content/dog.jpeg: 640x384 2 persons, 1 car, 1 dog, 1 handbag, 44.9ms
Speed: 2.8ms preprocess, 44.9ms inference, 340.3ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


**NOTE:** Result annotated image got saved in `{HOME}/runs/detect/predict/`. Let's display it.

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode


def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      // await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
import os
import pandas as pd
from PIL import Image
from ultralytics import YOLO

def crop_birds_from_folder(input_dir='images', output_dir='cropped_images', model_path='yolo11n.pt', conf_threshold=0.25):
    """
    Detects and crops birds from images in input_dir, saving all tax_id images into a single folder.

    :param input_dir: Root directory containing images (batch_n/tax_id/tax_id_imagenum.jpeg).
    :param output_dir: Root directory where cropped images will be saved (organized by tax_id).
    :param model_path: Path to the YOLO model.
    :param conf_threshold: Confidence threshold for detection.
    """
    model = YOLO(model_path)
    image_count = 0

    for root, _, files in os.walk(input_dir):
        path_parts = root.split(os.sep)
        if len(path_parts) < 3:
            continue  # Skip invalid directories (should be batch_n/tax_id)

        tax_id = path_parts[-1]  # Extract tax_id
        output_subdir = os.path.join(output_dir, tax_id)
        os.makedirs(output_subdir, exist_ok=True)

        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg')):
                image_count += 1
                print(f"Processing image {image_count}: {file}")
                image_path = os.path.join(root, file)

                try:
                    image = Image.open(image_path)
                    if image.width > 200 and image.height > 200:
                        result = model.predict(image, conf=conf_threshold, verbose=False)[0]

                        bird_detections = [
                            box for box, cls in zip(result.boxes.xyxy, result.boxes.cls) if cls == 14
                        ]  # Assuming class 14 is birds

                        for i, box in enumerate(bird_detections):
                            x_min, y_min, x_max, y_max = map(int, box)

                            if (x_max - x_min) >= 100 and (y_max - y_min) >= 100:
                                cropped_bird = image.crop((x_min, y_min, x_max, y_max))
                                file_name = f"{tax_id}_{os.path.splitext(file)[0]}_{i}.png"
                                file_path = os.path.join(output_subdir, file_name)
                                cropped_bird.save(file_path)
                except Exception as e:
                    print(f"Error processing image {image_count}: {file}. Skipping... Error: {e}")
                    continue

if __name__ == "__main__":
    crop_birds_from_folder()


Streaming output truncated to the last 5000 lines.
Processing image 45054: 4956_859.jpeg
Processing image 45055: 4956_956.jpeg
Processing image 45056: 4956_930.jpeg
Processing image 45057: 4956_990.jpeg
Processing image 45058: 4956_964.jpeg
Processing image 45059: 4956_970.jpeg
Processing image 45060: 4956_920.jpeg
Processing image 45061: 4956_913.jpeg
Processing image 45062: 4956_861.jpg
Processing image 45063: 4956_898.jpeg
Processing image 45064: 4956_949.jpg
Processing image 45065: 4956_881.jpeg
Processing image 45066: 4956_918.jpeg
Processing image 45067: 4956_917.jpeg
Processing image 45068: 4956_975.jpeg
Processing image 45069: 4956_858.jpg
Processing image 45070: 4956_944.jpg
Processing image 45071: 4956_885.jpg
Processing image 45072: 4956_922.jpeg
Processing image 45073: 4956_921.jpeg
Processing image 45074: 4956_927.jpeg
Processing image 45075: 4956_867.jpg
Processing image 45076: 4956_864.jpg
Processing image 45077: 4956_960.jpeg
Processing image 45078: 4956_1001.jpeg
Proce

KeyboardInterrupt: 

In [ ]:
!zip -r cropped_images.zip cropped_images/

Streaming output truncated to the last 5000 lines.
  adding: cropped_images/4665/4665_4665_100_0.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_107_0.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_37_0.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_31_0.png (stored 0%)
  adding: cropped_images/4665/4665_4665_63_0.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_67_0.png (stored 0%)
  adding: cropped_images/4665/4665_4665_53_0.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_93_0.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_182_0.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_164_0.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_177_0.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_103_0.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_176_0.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_192_1.png (deflated 0%)
  adding: cropped_images/4665/4665_4665_86_0.png (deflated 0%)


In [ ]:
!cp /content/cropped_images.zip /content/gdrive/MyDrive/
